# The Product Pricer Continued

A model that can estimate how much something costs, from its description.

## Enter The Frontier!

And now - we put Frontier Models to the test.

### 2 important points:

It's important to appreciate that we aren't Training the frontier models. We're only providing them with the Test dataset to see how they perform. They don't gain the benefit of the 400,000 training examples that we provided to the Traditional ML models.

HAVING SAID THAT...

It's entirely possible that in their monstrously large training data, they've already been exposed to all the products in the training AND the test set. So there could be test "contamination" here which gives them an unfair advantage. We should keep that in mind.

In [ ]:
!pip install datasets==3.6.0 gensim anthropic

In [ ]:
# imports

import os
import re
import math
import json
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
from collections import Counter
from openai import OpenAI
from anthropic import Anthropic
from google.colab import userdata

In [ ]:
# keys

openai_api_key = userdata.get("OPENAI_API_KEY")
claude_api_key = userdata.get("ANTHROPIC_API_KEY")
hugging_face_token = userdata.get("HF_TOKEN")

In [ ]:
# initialize

os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['ANTHROPIC_API_KEY'] = claude_api_key
os.environ['HF_TOKEN'] = hugging_face_token

In [44]:
# Log in to HuggingFace

login(hugging_face_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [46]:
# get the python files from github

%%bash
wget https://raw.githubusercontent.com/NormLorenz/ai-llm-google-colab/main/jupyter-notebooks/items.py
wget https://raw.githubusercontent.com/NormLorenz/ai-llm-google-colab/main/jupyter-notebooks/loaders.py
wget https://raw.githubusercontent.com/NormLorenz/ai-llm-google-colab/main/jupyter-notebooks/testing.py
wget https://raw.githubusercontent.com/NormLorenz/ai-llm-google-colab/main/jupyter-notebooks/human_input.py
wget https://raw.githubusercontent.com/NormLorenz/ai-llm-google-colab/main/jupyter-notebooks/human_output.py

--2025-10-28 17:37:39--  https://raw.githubusercontent.com/NormLorenz/ai-llm-google-colab/main/jupyter-notebooks/items.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3760 (3.7K) [text/plain]
Saving to: ‘items.py’

     0K ...                                                   100% 26.2M=0s

2025-10-28 17:37:39 (26.2 MB/s) - ‘items.py’ saved [3760/3760]

--2025-10-28 17:37:39--  https://raw.githubusercontent.com/NormLorenz/ai-llm-google-colab/main/jupyter-notebooks/loaders.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2855 

CalledProcessError: Command 'b'wget https://raw.githubusercontent.com/NormLorenz/ai-llm-google-colab/main/jupyter-notebooks/items.py\nwget https://raw.githubusercontent.com/NormLorenz/ai-llm-google-colab/main/jupyter-notebooks/loaders.py\n# wget https://raw.githubusercontent.com/NormLorenz/ai-llm-google-colab/main/jupyter-notebooks/testing.py\nwget https://raw.githubusercontent.com/NormLorenz/ai-llm-google-colab/main/jupyter-notebooks/human_input.py\nwget https://raw.githubusercontent.com/NormLorenz/ai-llm-google-colab/main/jupyter-notebooks/human_output.py\n'' returned non-zero exit status 8.

In [ ]:
# And some more imports - with thanks to Alex C for pointing out that I need to do these after the HF login..
# moved our Tester into a separate package
# call it with Tester.test(function_name, test_dataset)

from items import Item
from loaders import ItemLoader
from testing import Tester

In [ ]:
openai = OpenAI()
claude = Anthropic()

In [ ]:
%matplotlib inline

In [ ]:
# Mount your Google Drive at the beginning of the session
from google.colab import drive
drive.mount('/content/drive')

# Define the file path in your Google Drive
drive_path_one= '/content/drive/MyDrive/Colab Notebooks/train_lite.pkl'
drive_path_two= '/content/drive/MyDrive/Colab Notebooks/test_lite.pkl'

# Load the file
with open(drive_path_one, 'rb') as file:
    train = pickle.load(file)

# Load the file
with open(drive_path_two, 'rb') as file:
    test = pickle.load(file)

print("File loaded successfully:")

# Before we look at the Frontier

## There is one more model we could consider

In [ ]:
# Write the test set to a CSV

import csv
with open('human_input.csv', 'w', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    for t in test[:250]:
        writer.writerow([t.test_prompt(), 0])

In [ ]:
# Read it back in

human_predictions = []
with open('human_output.csv', 'r', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        human_predictions.append(float(row[1]))

In [ ]:
def human_pricer(item):
    idx = test.index(item)
    return human_predictions[idx]

In [ ]:
Tester.test(human_pricer, test)

## First, the humble but mighty GPT-4o-mini

It's called mini, but it packs a punch.

In [ ]:
# First let's work on a good prompt for a Frontier model
# Notice that I'm removing the " to the nearest dollar"
# When we train our own models, we'll need to make the problem as easy as possible,
# but a Frontier model needs no such simplification.

def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [ ]:
# Try this out

messages_for(test[0])

In [ ]:
# A utility function to extract the price from a string

def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [ ]:
get_price("The price is roughly $99.99 because blah blah")

In [ ]:
# The function for gpt-4o-mini

def gpt_4o_mini(item):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(item),
        seed=42,
        max_tokens=5
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
test[0].price

In [ ]:
Tester.test(gpt_4o_mini, test)

In [ ]:
def gpt_4o_frontier(item):
    response = openai.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=messages_for(item),
        seed=42,
        max_tokens=5
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
# The function for gpt-4o - the August model
# Note that it cost me about 1-2 cents to run this (pricing may vary by region)
# You can skip this and look at my results instead

Tester.test(gpt_4o_frontier, test)

In [ ]:
def claude_3_point_5_sonnet(item):
    messages = messages_for(item)
    system_message = messages[0]['content']
    messages = messages[1:]
    response = claude.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=5,
        system=system_message,
        messages=messages
    )
    reply = response.content[0].text
    return get_price(reply)

In [ ]:
# The function for Claude 3.5 Sonnet
# It also cost me about 1-2 cents to run this (pricing may vary by region)
# You can skip this and look at my results instead

Tester.test(claude_3_point_5_sonnet, test)